In [6]:
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from PIL import Image
from tqdm import tqdm
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.metrics import accuracy_score

# Путь для сохранения и загрузки весов модели
weights_path = "my_trained_model.pth"
# Проверочный файл
image_path = "gru.jpg"
# Путь для тестов
test_dataset_path = "./datasets/PetImages/"
# Путь для сохранения и загрузки данных
dataset_path = "./datasets/PetImages/"
# Путь для загрузки весов
download_path = "./datasets"
# откуда/куда фотки подгружаем 
zip_filename = "microsoft-catsvsdogs-dataset.zip"
# количество эпох
num_epochs = 2
# количество классов
num_classes = 2
# откуда берем данные
dataset_name = "shaunthesheep/microsoft-catsvsdogs-dataset"

# Проверка наличия тестовых данных, если его нет, скачиваем
if not os.path.exists(dataset_path):
    api = KaggleApi()
    # аутентификация по json
    api.authenticate()
    # Скачиваем архив данных
    api.dataset_download_files(dataset_name, path=download_path)

    # Распаковываем архив
    with zipfile.ZipFile(os.path.join(download_path, zip_filename), "r") as zip:
        zip.extractall(download_path)

# Загрузить сохраненную модель, если она существует
if os.path.exists(weights_path):
    # не обученная
    loaded_model = models.vgg19(pretrained=False)
    # 6 - последний слой
    loaded_model.classifier[6] = nn.Linear(4096, num_classes)
    loaded_model.eval()
    print("Готовая модель загружена.")
else:
    # Обучение модели, если сохраненная модель отсутствует
    print("Старт обучения модели.")
    # последовательность преобразований данных обучения
    transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # подготовленные изображения
    train_dataset = ImageFolder(root=dataset_path, transform=transform_train)
    val_size = int(0.2 * len(train_dataset))
    train_size = len(train_dataset) - val_size
    train_dataset, test_dataset = random_split(train_dataset, [train_size, val_size])
    # данные для тренировки
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    # Загрузка тестовых данных
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=1)

    # Определение модели VGG-19
    model = models.vgg19(pretrained=True)
    # Замораживаем параметры предварительно обученной модели
    for param in model.parameters():
        param.requires_grad = False

    # Изменяем последний слой, чтобы он соответствовал числу классов в вашем наборе данных
    model.classifier[-1] = torch.nn.Linear(4096, num_classes)

    # Обучение модели
  
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Определение функции потерь и оптимизатора
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


    for epoch in range(num_epochs):
        print(f"Эпоха: {epoch + 1}")
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Сохранить веса обученной модели
    torch.save(model.state_dict(), weights_path)
    print("Обучение завершено. Веса модели сохранены.")

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

image = Image.open(image_path)
image = transform_test(image).unsqueeze(0) 

# Оценка качества модели на тестовом изображении
loaded_model.eval()
with torch.no_grad():
    if os.path.exists(weights_path):
        output = loaded_model(image)
    else:
        output = model(image)

# Получаем индекс класса с наибольшей вероятностью
_,predicted_class = torch.max(output, 1)

# Выводим имя класса
classes = os.listdir(dataset_path)
predicted_class_name = classes[predicted_class.item()]
print("Возможный класс объекта:", predicted_class_name)

# Загрузка данных для тестирования
val_size = int(0.2 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, test_dataset = random_split(train_dataset, [train_size, val_size])
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=1)

# Оценка качества модели на тестовых данных
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Точность модели на тестовых данных: {accuracy * 100:.2f}%')

Старт обучения модели.
Эпоха: 1
Эпоха: 2
Обучение завершено. Веса модели сохранены.
Возможный класс объекта: Dog
Точность модели на тестовых данных: 98.70%
